### Association rules by HouseHold
#### Department - HouseHold

In [91]:
##importing libraries
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import OnehotTransactions
import pandas as pd
from sklearn import cluster, datasets
import numpy as np
import scipy
from scipy.cluster.vq import kmeans,vq
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

In [92]:
# Csv contains data of trainorders and prior orders
combine_train_prior=pd.read_csv("CombineOrders1.csv",encoding = "ISO-8859-1")

In [93]:
## altering datatypes for ease of use
combine_train_prior['aisle_id']=combine_train_prior['aisle_id'].astype('int16')
combine_train_prior['order_id']=combine_train_prior['order_id'].astype('int32')
combine_train_prior['product_id']=combine_train_prior['product_id'].astype('int32')
combine_train_prior['add_to_cart_order']=combine_train_prior['add_to_cart_order'].astype('int16')
combine_train_prior['reordered']=combine_train_prior['reordered'].astype('int8')
combine_train_prior['department_id']=combine_train_prior['department_id'].astype('int8')



In [94]:
combine_train_prior['product_name'] = combine_train_prior['product_name'].str.strip()
combine_train_prior.dropna(axis=0, subset=['order_id'], inplace=True)

In [95]:
## this will give us the popular departments as departments with the highest value counts will be those from where consumers have purchased the most
##Top 5 Departments are produce,dairy eggs, snacks, beverages, frozen
combine_train_prior['department'].value_counts()

produce            9888378
dairy eggs         5631067
snacks             3006412
beverages          2804175
frozen             2336858
pantry             1956819
bakery             1225181
canned goods       1114857
deli               1095540
dry goods pasta     905340
household           774652
meat seafood        739238
breakfast           739069
personal care       468693
babies              438743
international       281155
alcohol             159294
pets                102221
missing              77396
other                38086
bulk                 35932
Name: department, dtype: int64

In [96]:
combine_train_prior

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,1,1,43633,5,1,112108,train,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods
2,2,1,10246,3,0,112108,train,4,4,10,9.0,Organic Celery Hearts,83,4,fresh vegetables,produce
3,3,1,22035,8,1,112108,train,4,4,10,9.0,Organic Whole String Cheese,21,16,packaged cheese,dairy eggs
4,4,1,47209,7,0,112108,train,4,4,10,9.0,Organic Hass Avocado,24,4,fresh fruits,produce
5,5,1,13176,6,0,112108,train,4,4,10,9.0,Bag of Organic Bananas,24,4,fresh fruits,produce
6,6,1,11109,2,1,112108,train,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs
7,7,1,49683,4,0,112108,train,4,4,10,9.0,Cucumber Kirby,83,4,fresh vegetables,produce
8,8,36,46979,8,1,79431,train,23,6,18,30.0,Asparagus,83,4,fresh vegetables,produce
9,9,36,39612,1,0,79431,train,23,6,18,30.0,Grated Pecorino Romano Cheese,2,16,specialty cheeses,dairy eggs


In [97]:
## Using Apriori we will find if there are association rules found for products bought from department Dairy Eggs
dept_produce=combine_train_prior[combine_train_prior['department'] =="household"].reset_index()

In [98]:
## Preparing dataset for Apriori
temp_df=dept_produce.loc[:,['order_id','product_name']]

In [99]:
## In order to find association we have considered those orders that have products which 4 or more in number
grouped_data=temp_df.groupby("order_id").filter(lambda x: len(x) >=3)

In [100]:
grouped_data

,order_id,product_name
0,98,Plastic Wrap
1,98,Aluminum Foil
2,98,Lavender Scent Laundry Detergent
3,98,Plastic Spoons
4,98,Geranium Liquid Dish Soap
5,226,Wrap Aluminum Foil Heavy Duty
6,226,Choose-A-Size White Paper Towels 8 Giant Rolls
7,226,Soft White 60 Watts Halogen Bulb
30,3321,Heavy Duty Scrub Sponge
31,3321,Variety Pack


In [101]:
## Pivoting the table to get occurences of a product in a particuler order
from pandas import *
# table=pd.DataFrame()
table1=pivot_table(grouped_data,index='order_id',columns='product_name',aggfunc=len)

In [102]:
table1

product_name,#2 Mechanical Pencils,1 Cup Measuring Cup,1 Ply Napkins,1 Ply Paper Towels,1 Ply White Luncheon Napkins,10 Inch Wheat Straw Plates,10 oz Paper Bowls,"10.25\"" Elegant Fluted Party Plates",100 % Recycled Paper Towels,100% Beeswax Hand Dipped Tapers,...,Zipper Sandwich Bags,e-Tronic Wipes,"flings! Laundry Detergent Pacs, Original, 57 Count Laundry",flings! Original Laundry Detergent Pacs,iChef Casserole Pans with Lids (10 7/16 in x 8 in x 1 3/4 in),with Bleach Disinfectant Cleanser Scratch Free Lavender Fresh,with Bleach Powder Cleanser,with Color Safe Brightener Power Paks 2in1 Stain Fighter,with Dawn Action Pacs Fresh Scent Dishwasher Detergent Pacs,with Twist Ties Sandwich & Storage Bags
order_id,,,,,,,,,,,,,,,,,,,,,
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
#relace NAN with 0
table1=table1.fillna(0)

In [104]:
## of the product exists then value changed to 1 else 0
def encode_units(x):
    if x == 0.0:
        return 0
    if x >= 1.0:
        return 1
basket_sets = table1.applymap(encode_units)

In [105]:
## Running Apriori to get frequent itemset with a minimum suppirt of 0.01. ie the combination must exist atleast in 1% of dataset
frequent_itemsets = apriori(table1, min_support=0.01, use_colnames=True)

In [106]:
frequent_itemsets

,support,itemsets
0,0.013614,[1 Ply Paper Towels]
1,0.010863,[100% Recycled 2 Ply Jumbo Paper Towel Roll]
2,0.010863,[100% Recycled Bath Tissue Rolls]
3,0.018991,[100% Recycled Bathroom Tissue]
4,0.049682,[100% Recycled Paper Towels]
5,0.010136,"[12\"" Aluminium Foil]"
6,0.010831,[13 Gallon Tall Drawstring Kitchen Bags]
7,0.012128,[9 Inch Plates]
8,0.014168,[All Purpose Cleaner]
9,0.038060,[Aluminum Foil]


In [107]:
# Getting the rules of association based on the minimum support selected
rules = association_rules(frequent_itemsets, metric="lift",min_threshold=1)

In [108]:
rules

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(100% Recycled Paper Towels),(Sustainably Soft Bath Tissue),0.049682,0.032810,0.013978,0.281349,8.574989,0.012348,1.345841
1,(Sustainably Soft Bath Tissue),(100% Recycled Paper Towels),0.032810,0.049682,0.013978,0.426024,8.574989,0.012348,1.655675


In [109]:
## Since, ideally Lift must greater than 1 , we have selected the rules to have Lift to be greater than 1, confidence greater than 30% and a conviction of 40%
final_result=rules[ (rules['lift'] > 1) &
       (rules['confidence'] >= 0.3) & rules['conviction']<1.4 ]
final_result

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(100% Recycled Paper Towels),(Sustainably Soft Bath Tissue),0.049682,0.032810,0.013978,0.281349,8.574989,0.012348,1.345841
1,(Sustainably Soft Bath Tissue),(100% Recycled Paper Towels),0.032810,0.049682,0.013978,0.426024,8.574989,0.012348,1.655675


In [110]:
final_result.to_csv("Department_householdApriori.csv",sep=',')

In [111]:
test1=pd.read_csv("Department_householdApriori.csv")

In [112]:
## Cleaning the resultset for usability
test1=test1.replace({'frozenset': ''}, regex=True)

In [113]:
test1=test1.replace({"u'": ''}, regex=True)

In [114]:
test1['antecedants'] = test1['antecedants'].str.strip('()').str.strip('[]')
test1['antecedants'] = test1['antecedants'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['antecedants'] = test1['antecedants'].str.replace("'",'')

In [115]:
test1['consequents'] = test1['consequents'].str.strip('()')
test1['consequents'] = test1['consequents'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['consequents'] = test1['consequents'].str.replace("'",'')

In [116]:
test1=test1.drop(['Unnamed: 0'], axis = 1)

In [117]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,100% Recycled Paper Towels,Sustainably Soft Bath Tissue,0.049682,0.032810,0.013978,0.281349,8.574989,0.012348,1.345841
1,Sustainably Soft Bath Tissue,100% Recycled Paper Towels,0.032810,0.049682,0.013978,0.426024,8.574989,0.012348,1.655675


In [118]:
## Writing results to a csv file
test1.to_csv("FinalDepartment_householdApriori.csv")

In [119]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,100% Recycled Paper Towels,Sustainably Soft Bath Tissue,0.049682,0.032810,0.013978,0.281349,8.574989,0.012348,1.345841
1,Sustainably Soft Bath Tissue,100% Recycled Paper Towels,0.032810,0.049682,0.013978,0.426024,8.574989,0.012348,1.655675


In [120]:
## testing the result set
t=test1[test1['antecedants'] =="Organic Strawberries, Organic Baby Spinach"].reset_index()